## Import libraries

In [2]:
import cv2
import os
import shutil
import mediapipe as mp
import numpy as np # Za np.zeros_like, če bi bil potrebno
from tqdm import tqdm

In [3]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
# Ali še bolj specifično za Metal:
print(tf.config.list_physical_devices('GPU'))

Num GPUs Available:  1
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [4]:
print(f"MediaPipe verzija: {mp.__version__}")
print(f"OpenCV verzija: {cv2.__version__}")

MediaPipe verzija: 0.10.21
OpenCV verzija: 4.11.0


In [5]:
# Model_selection=1 je za zaznavanje obrazov na razdalji (celotni obraz), bolj robustno
# min_detection_confidence=0.7 pomeni, da je potrebna visoka zanesljivost za zaznavo
mp_face_detection = mp.solutions.face_detection
face_detection = mp_face_detection.FaceDetection(model_selection=1, min_detection_confidence=0.7)

I0000 00:00:1748726596.415888 7030604 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M4 Pro
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [6]:
# --- Parametri ---
TARGET_IMG_SIZE = (224, 224) # Željena velikost izrezanih obrazov
MARGIN_FACTOR = 1         # Faktor za razširitev izrezanega okvirja obraza (1.0 = samo obraz)

In [7]:
# Pot do mape, kjer so shranjene podmape z LFW slikami (npr. 'Aaron_Eckhart/', 'Abel_Pacheco/')
# PREVERI TO POT NA SVOJEM SISTEMU!
# Glede na tvoj tree diagram, je verjetno to:
INPUT_LFW_RAW_FOLDER = '../data/raw/Ifw-dataset/lfw-deepfunneled/lfw-deepfunneled'

# Pot do mape, kamor bomo shranili obdelane "false data" obraze
OUTPUT_PROCESSED_FALSE_DATA_FOLDER = '../data/processed/false_data'

In [8]:
# --- Funkcija za izrezovanje in prilagoditev obraza ---
def crop_face_only(image, detection, target_size=TARGET_IMG_SIZE, margin_factor=MARGIN_FACTOR):
    h, w, _ = image.shape
    location_data = detection.location_data
    relative_bounding_box = location_data.relative_bounding_box

    # Izračun absolutnih koordinat okvirja
    xmin = int(relative_bounding_box.xmin * w)
    ymin = int(relative_bounding_box.ymin * h)
    box_width = int(relative_bounding_box.width * w)
    box_height = int(relative_bounding_box.height * h)

    xmax = xmin + box_width
    ymax = ymin + box_height

    # Izračun centra okvirja
    center_x = (xmin + xmax) / 2
    center_y = (ymin + ymax) / 2

    # Določitev največje dimenzije in dodajanje marže
    max_dim = max(box_width, box_height) * margin_factor
    
    # Izračun novih koordinat izrezanega okvirja z maržo
    x1 = int(center_x - max_dim / 2)
    y1 = int(center_y - max_dim / 2)
    x2 = int(center_x + max_dim / 2)
    y2 = int(center_y + max_dim / 2)

    # Poskrbimo, da okvir ne preseže meja slike
    x1 = max(0, x1)
    y1 = max(0, y1)
    x2 = min(w, x2)
    y2 = min(h, y2)

    # Izrezovanje obraza
    cropped_face = image[y1:y2, x1:x2]
    
    # Preverimo, če je izrezan obraz prazen (lahko se zgodi, če so koordinate napačne)
    if cropped_face.shape[0] == 0 or cropped_face.shape[1] == 0:
        return None

    # Sprememba velikosti izrezanega obraza na ciljno velikost
    final_face = cv2.resize(cropped_face, target_size, interpolation=cv2.INTER_AREA)
    return final_face

In [9]:
# --- Priprava izhodne mape ---
# Preverimo, ali izhodna mapa že obstaja, in jo po potrebi izbrišemo/ustvarimo
if os.path.exists(OUTPUT_PROCESSED_FALSE_DATA_FOLDER):
    print(f"Izhodna mapa '{OUTPUT_PROCESSED_FALSE_DATA_FOLDER}' že obstaja. Brišem jo in ustvarjam novo.")
    shutil.rmtree(OUTPUT_PROCESSED_FALSE_DATA_FOLDER)
os.makedirs(OUTPUT_PROCESSED_FALSE_DATA_FOLDER, exist_ok=True)

# --- Glavni proces obdelave LFW slik ---
total_processed_faces = 0
total_images_skipped = 0

print(f"Začenjam z obdelavo LFW slik iz: {INPUT_LFW_RAW_FOLDER}")

Izhodna mapa '../data/processed/false_data' že obstaja. Brišem jo in ustvarjam novo.


W0000 00:00:1748726596.429555 7035991 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Začenjam z obdelavo LFW slik iz: ../data/raw/Ifw-dataset/lfw-deepfunneled/lfw-deepfunneled


In [10]:
# Pridobi seznam vseh map oseb v LFW
person_folders = [f for f in os.listdir(INPUT_LFW_RAW_FOLDER) if os.path.isdir(os.path.join(INPUT_LFW_RAW_FOLDER, f))]

# Zanka skozi osebe z uporabo tqdm za progress bar
# Desc je opis, ki se prikaže ob progress baru
with tqdm(total=len(person_folders), desc="Obdelava LFW obrazov") as pbar:
    for person_folder_name in person_folders:
        person_folder_path = os.path.join(INPUT_LFW_RAW_FOLDER, person_folder_name)

        face_processed_for_this_person = False

        image_files_in_folder = [f for f in os.listdir(person_folder_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
        image_files_in_folder.sort()

        for image_name in image_files_in_folder:
            if face_processed_for_this_person:
                break

            image_path = os.path.join(person_folder_path, image_name)

            image_bgr = cv2.imread(image_path)

            if image_bgr is None:
                # Opozorila ne izpisujemo za vsako sliko, da ne spamamo konsolo.
                # Štejemo samo, da je bila preskočena.
                total_images_skipped += 1
                continue

            image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)

            results = face_detection.process(image_rgb)

            if results.detections:
                processed_face_rgb = crop_face_only(image_rgb, results.detections[0], target_size=TARGET_IMG_SIZE)

                if processed_face_rgb is None:
                    total_images_skipped += 1
                    continue

                processed_face_bgr = cv2.cvtColor(processed_face_rgb, cv2.COLOR_RGB2BGR)

                unique_filename = f"{person_folder_name}.jpg"
                output_path = os.path.join(OUTPUT_PROCESSED_FALSE_DATA_FOLDER, unique_filename)

                cv2.imwrite(output_path, processed_face_bgr)
                total_processed_faces += 1
                face_processed_for_this_person = True
            else:
                total_images_skipped += 1
                
        # Po obdelavi vsake osebe posodobimo progress bar
        pbar.update(1)
        
        # Opozorilo za osebe, ki niso bile obdelane, prikažemo samo ob koncu, če jih je veliko
        # ali se poskusimo izogniti preveč izpisom, saj tqdm že kaže napredek
        # Lahko pa pustiš tudi ta izpis, če želiš videti takojšnje opozorilo
        # if not face_processed_for_this_person:
        #     print(f"Opozorilo: Za osebo '{person_folder_name}' ni bilo mogoče obdelati nobene slike.")


print(f"\nKončana obdelava LFW dataset-a. Skupaj obdelanih in shranjenih {total_processed_faces} obrazov v: {OUTPUT_PROCESSED_FALSE_DATA_FOLDER}")
print(f"Skupaj preskočenih slik (napake pri nalaganju/zaznavi/obrezovanju): {total_images_skipped}")

face_detection.close()

Obdelava LFW obrazov: 100%|███████████████████████████████████████████████████████████████████████████████| 5749/5749 [00:17<00:00, 336.87it/s]


Končana obdelava LFW dataset-a. Skupaj obdelanih in shranjenih 5651 obrazov v: ../data/processed/false_data
Skupaj preskočenih slik (napake pri nalaganju/zaznavi/obrezovanju): 141
